In [1]:
from cil.io import NEXUSDataReader, NEXUSDataWriter
from cil.optimisation.algorithms import SPDHG
from cil.plugins.astra.operators import ProjectionOperator
from cil.optimisation.functions import IndicatorBox, L2NormSquared, L1Norm, MixedL21Norm, BlockFunction
from cil.optimisation.operators import GradientOperator, BlockOperator
from cil.framework import BlockDataContainer, AcquisitionGeometry
from cil.processors import Slicer
from cil.optimisation.operators import GradientOperator

In [2]:
# Load data after the RingRemover processor
name = "data_after_ring_remover_318_398.nxs"
read_data = NEXUSDataReader(file_name = "HyperspectralData/" + name)
data = read_data.load_data()
print("Sinogram shape is {}".format(data.shape))

Sinogram shape is (80, 80, 120, 400)


# Select only 5 energy channels and 5 vertical slices. 

# To reproduce the results in the paper, comment the line below. The reconstruction with 10 subsets takes about 6 hours.

In [3]:
# comment to reproduce the results in the paper
data = Slicer(roi={'channel': (37,42),'vertical': (17,22)})(data)
print("Sinogram shape is {}".format(data.shape))

# Extract geometry and angle information from data
ag = data.geometry
ig = ag.get_ImageGeometry()
angles = ag.angles

Sinogram shape is (5, 5, 120, 400)


In [4]:
# Setup SPDHG parameters
sample = "stride"

# Choose number of subsets
subsets = 10
size_of_subsets = int(len(angles)/subsets)

if sample =="uniform":
    list_data = [Slicer(roi={'angle':(i,len(angles),i+size_of_subsets)})(data) for i in range(0, len(angles), size_of_subsets)]
elif sample =="stride":  
    list_data = [Slicer(roi={'angle':(i,len(angles),subsets)})(data) for i in range(subsets)]
            
list_geometries = [ag.geometry for ag in list_data ]

Grad = GradientOperator(ig, correlation="SpaceChannels", split=True)

# For every geometry in list_geometries, define operators A_i
A_i = []
A_i = [ProjectionOperator(ig, ageom, 'gpu') for ageom in list_geometries]
A_i.append(Grad)

# Wrap the projection operators A_i and GradientOperator to the BlockOperator K
K = BlockOperator(*A_i)

b = BlockDataContainer(*list_data)
    
# List of probabilities
prob = [1/(2*subsets)]*(subsets) + [1/2]   

# Regularisation parameter
alpha = 0.001
beta = 0.2
   
# List of BlockFunctions
fsubsets = [0.5*L2NormSquared(b = b[i]) for i in range(subsets)]

# Setup (3D+spectral)TV term using BlockFunction
f = BlockFunction(*fsubsets, BlockFunction(beta * L1Norm(), alpha * MixedL21Norm()))

# Positivity constraint of g
g = IndicatorBox(lower=0)

Initialised GradientOperator with C backend running with  16  threads


In [5]:
# Run SPDHG algorithm
spdhg = SPDHG(f = f, g = g, operator = K, 
              max_iteration = 1001,
              update_objective_interval = 200, prob = prob )
spdhg.run(verbose=0) 

SPDHG setting up
SPDHG configured


/home/vaggelis/miniconda3/envs/cil21_tigre_astra/lib/python3.8/site-packages/cil/framework/framework.py:2185: RuntimeWarning: invalid value encountered in true_divide
  pwop(self.as_array(), x2.as_array(), *args, **kwargs )


In [6]:
name = "spdhg_3d_spectral_tv_reconstruction.nxs"
writer = NEXUSDataWriter(file_name = "HyperspectralData/" + name,
                         data = spdhg.solution)     
writer.write() 